# Part 1: Fundamentals of Stress, Strain, and Mechanical Behavior of Materials

## Introduction

In the realm of materials science and mechanics, comprehending how materials react to external forces is paramount. This comprehension forms the foundation for predicting material behavior under various circumstances, which is indispensable for engineering design and analysis. In this section, we will revisit the core concepts of stress, strain, and the mechanical behavior of materials.

## Stress and Strain

**Stress** is a pivotal mechanical property that characterizes a material's internal resistance to deformation when exposed to external forces. It is defined as the force per unit area and signifies the intensity of the internal forces within a material. Stress serves as an indicator of the mechanical load borne by a material and is a critical parameter for assessing its strength and stability.

**Strain**, on the other hand, gauges the amount of deformation a material undergoes in response to applied forces or loads. It quantifies the alteration in size or shape of a material relative to its original dimensions. Strain is typically expressed as the ratio of the change in dimension to the original dimension.

The equations for stress and strain are as follows:

* Stress:

$(\( \sigma \)) = \( \frac{Force (F)}{Area (A)} \)$

* Strain:

$(\( \epsilon \)) = \( \frac{Change\,in\,Length (\Delta L)}{Original\,Length (L)} \)$

## Stress-Strain Relationship

The interconnection between stress and strain is fundamental to grasping how materials perform under load. This relationship is conventionally depicted using a **stress-strain diagram**, a graphical representation of a material's response to escalating stress. The stress-strain diagram provides insights into a material's elastic behavior, yield point, plastic deformation, and ultimate failure.

![Stress-Strain Diagram](output.png)

## Hooke's Law

One of the primary principles governing the elasticity of materials is **Hooke's Law**, which asserts that the deformation (strain) of a material is directly proportional to the applied stress within the elastic limit. Mathematically, Hooke's Law can be formulated as:

\[ \sigma = E \cdot \epsilon \]

where:
- \( \sigma \) represents the stress applied to the material.
- \( E \) denotes the Young's Modulus or Elastic Modulus, a material property indicating its stiffness.
- \( \epsilon \) signifies the strain experienced by the material.

Hooke's Law is a fundamental concept for comprehending the initial behavior of materials within their elastic range. Beyond this point, materials undergo plastic deformation and might not return to their original state upon unloading.

## Conclusion

In this section, we have revisited the core notions of stress, strain, and the mechanical behavior of materials. These notions are indispensable for understanding how materials respond to external forces and loads. Moreover, they form the bedrock for delving into more advanced subjects in the study of deformable bodies and the mechanics of materials.


In [27]:
import numpy as np
import math
from scipy.interpolate import interp1d
import plotly.graph_objects as go

In [28]:
# define your values
Force = [0, 4400, 5670, 6600, 7040, 8320, 8800, 9200, 9240]
Elongation = [0, 1, 2, 4, 5, 8, 10, 15, 18]
GLength = 75
Radius = 6.65
LinearLimit = 1

# area calculation
Area = math.pi * Radius * Radius

# stress strain calculated as per above values
Stress = [x * 9.8 / Area for x in Force]
Strain = [x / GLength for x in Elongation]

# True Stress calculation
Stress_True = [x * (1 + y) for y, x in zip(Strain, Stress)]

# True Strain calculation
Strain_True = [math.log(1 + x) for x in Strain]

# values created which are needed for interpolation
Strain_values_linear = np.linspace(Strain[0], Strain[LinearLimit], num=41, endpoint=True)
Strain_values_eng = np.linspace(Strain[LinearLimit], Strain[-1], num=41, endpoint=True)
Strain_values_true = np.linspace(Strain_True[LinearLimit], Strain_True[-1], num=41, endpoint=True)

# interpolation defined
f1 = interp1d(Strain, Stress, fill_value='extrapolate')
f2 = interp1d(Strain, Stress, kind=3, fill_value='extrapolate')
f3 = interp1d(Strain_True, Stress_True, kind=3)




In [29]:
# find offset yield value
val = Strain[LinearLimit]
step = (Strain[LinearLimit + 1] - Strain[LinearLimit]) / 50
sign = f4(val) - f2(val)
while True:
    if (f4(val) - f2(val)) * sign < 0:
        break
    val = val + step
YieldPoint = f4(val - step)

In [32]:
# create Plotly figure
fig = go.Figure()

# plot data and interpolated lines
fig.add_trace(go.Scatter(x=Strain, y=Stress, mode='markers', name='Actual Values'))
fig.add_trace(go.Scatter(x=Strain_values_linear, y=f1(Strain_values_linear), mode='lines', name='Linear Region'))
fig.add_trace(go.Scatter(x=Strain_values_eng, y=f2(Strain_values_eng), mode='lines', name='Engineering Stress Strain'))
fig.add_trace(go.Scatter(x=Strain_values_true, y=f3(Strain_values_true), mode='lines', name='True Stress Strain'))

# plot interp1d point
interp1d_point = go.Scatter(x=[Strain[LinearLimit]], y=[Stress[LinearLimit]], mode='markers', name='interp1d')
fig.add_trace(interp1d_point)

# plot UTS Line
fig.add_shape(dict(type="line", x0=min(Strain), x1=max(Strain), y0=uts, y1=uts, line=dict(color='cyan', dash='dash')))

# plot Ductility Line
fig.add_shape(dict(type="line", x0=Strain[-1], x1=Strain[-1], y0=0, y1=max(Stress), line=dict(color='purple', dash='dash')))

# plot offset Line
xA = [Strain[0], Strain[LinearLimit]]
yA = [Stress[0], Stress[LinearLimit]]
Strain_values_offset = [x + 0.002 * Strain[-1] for x in xA]
f4 = interp1d(Strain_values_offset, yA, fill_value='extrapolate')
Strain_values_offset.append(Strain[LinearLimit + 1])
fig.add_trace(go.Scatter(x=Strain_values_offset, y=f4(Strain_values_offset), mode='lines', name='Offset Line', line=dict(color='orange', dash='dash')))

# plot offset horizontal line
fig.add_shape(dict(type="line", x0=min(Strain), x1=max(Strain), y0=YieldPoint, y1=YieldPoint, line=dict(color='black', dash='dash')))

# add annotations
annotations = [
    dict(x=Strain[-1], y=max(Stress), xref="x", yref="y", text="Max Strain", showarrow=True, arrowhead=2),
    dict(x=min(Strain), y=uts, xref="x", yref="y", text="UTS", showarrow=True, arrowhead=2),
    dict(x=Strain[-1], y=YieldPoint, xref="x", yref="y", text="Yield Point", showarrow=True, arrowhead=2),
    dict(x=Strain[LinearLimit], y=Stress[LinearLimit], xref="x", yref="y", text="interp1d", showarrow=True, arrowhead=2)
]

# add anchored text
anchored_text = f"Young's Modulus = {Modulus:.5f} GPa<br>" \
                f"Elastic Limit = {ElasticLimit:.5f} MPa<br>" \
                f"Yield Stress = {YieldPoint:.5f} MPa<br>" \
                f"UTS = {uts:.5f} MPa<br>" \
                f"Failure Stress = {failure_stress:.5f} MPa<br>" \
                f"Max Strain = {Strain[-1]:.5f}"
fig.add_annotation(go.layout.Annotation(x=0.85, y=0.95, xref="paper", yref="paper", text=anchored_text, showarrow=False))

# layout settings
fig.update_layout(
    xaxis_title="Strain",
    yaxis_title="Stress (MPa)",
    xaxis_range=[0, max(Strain)],
    yaxis_range=[0, max(Stress)],
    showlegend=True,
    legend=dict(x=1, y=1),
    paper_bgcolor='black',            # Set background color to black
    plot_bgcolor='black',             # Set plot area color to black
    font=dict(color='white')          # Set font color to white
)

# set figure size
fig.update_layout(width=800, height=600)

# display graph
fig.show()


# Part 2 : We will study a experimental srtess-strain

:## Problem 1 - Analysis of experimental srtess-strain curves of alluminum 6061-T651

We are going to analyze the dataset collected by {cite}`AAKASH2019104085`.
They perform two types of experiments.
We are going to focus on the "uniaxial tension experiment." 
This is what it is all about:
+ They take several speciments of alluminum 6061-T651 ([a high-temperature alluminum alloy](https://en.wikipedia.org/wiki/6061_aluminium_alloy)).
+ The shape of these speciments is like this:

```{figure} https://ars.els-cdn.com/content/image/1-s2.0-S2352340919304391-gr1_lrg.jpg
---
height: 300px
name: directive-fig
---
Geometry of the uniaxial tension specimen.
```
+ They mounted the speciment (using the holes you see on the sides) on a machine that applied tension:

```{figure} https://ars.els-cdn.com/content/image/1-s2.0-S2352340919304391-gr5_lrg.jpg
---
height: 300px
name: directive-fig2
---
The mounted specimen.
```
+ They controlled the temperature of the specimen.
+ They applied tension gradually until the speciment broke, recorging at each step the strain (% change in length) and the stress (force per cross section area in MPa - megaPascal).

I suggest that you skim through the paper if you want to understand more about the details of the experiment.
Measuring the strains and stresses is not as straightforward as it sounds...

Our goal is to download the data, and for a fixed temperature, create a model for the stress-strain relation.

In [ ]:
url = 'https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/rd6jm9tyb6-2.zip'
download(url)
!unzip -o rd6jm9tyb6-2.zip
!unzip -o stress-strain-curves-of-aluminum-6061-t651-from-9-lots-at-6-temperatures-under-uniaxial-and-plain-strain-tension.zip